In [1]:
#Importing libraries
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam, RMSprop
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D
from tensorflow.keras import Model
import PIL
#Pretrained model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#FEEDING DATA INTO THE MODEL
from tensorflow.keras.preprocessing.image import ImageDataGenerator #allows us to augment and process data fed into a CNN

C:\Users\USER2\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\USER2\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\USER2\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\USER2\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [2]:
path_inception = "D:/amitavsir/bcd/data/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
local_weights_file = path_inception
pre_trained_model = InceptionV3(input_shape = (121, 121, 3), 
                                include_top = False, 
                                weights = None)
pre_trained_model.load_weights(local_weights_file)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [3]:
#Locking the weights and parameters of pretrained model
for layer in pre_trained_model.layers:
    layer.trainable = False

#Getting desired layer output
last_layer = pre_trained_model.get_layer('mixed8')
last = last_layer.output
print('Last layer output shape : ', last.shape)

Last layer output shape :  (?, 2, 2, 1280)


In [4]:
from tensorflow.keras.losses import categorical_crossentropy
# Model configuration
batch_size = 50
img_size, img_num_channels = 121, 3
loss_function = categorical_crossentropy
no_classes = 4
no_epochs = 15
optimizer = Adam()
verbosity = 1

In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import os,cv2,random
import numpy as np
import pandas as pd

In [8]:
data_path='D:/amitavsir/bcd/datacompressed/train'
list_folder=os.listdir(path = data_path)
data=[]
im_size=121    
for i in list_folder:
    new_path=os.path.join(data_path,i) 
    pic_list=os.listdir(new_path)                                               
    for img in pic_list:
        pic=os.path.join(new_path,img)   
        arr=cv2.imread(pic)    
        data.append([arr,list_folder.index(i)])    
        
random.shuffle(data)  
x_train,y_train=[],[]
for i,j in data:
    x_train.append(i)
    y_train.append(j)
x_train=np.array(x_train).reshape(-1,im_size,im_size,3)
y_train=np.array(y_train).reshape(-1,1)

encoder = OneHotEncoder()
y_train = encoder.fit_transform(y_train)
X_train = x_train/255
y_train = y_train.toarray()

X_train, X_test, Y_train, Y_test = train_test_split(X_train,y_train, test_size=0.25)

In [9]:
# Parse numbers as floats
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [10]:
# Define per-fold score containers <-- these are new
acc_per_fold = [] #stores accuracy
loss_per_fold = [] #stores losses

In [11]:
# Merge inputs and targets
inputs = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((Y_train, Y_test), axis=0)

In [12]:
#FEEDING DATA INTO THE MODEL
from tensorflow.keras.preprocessing.image import ImageDataGenerator #A tool that allows us to augment and process data to be fed into a CNN

#class for training set
train_data = ImageDataGenerator(rescale = 1./255, #pixel values rescaled so that it stays between 0 and 1.
                                shear_range = 0.2, #this and the next 4 steps are used to augment our data by shearing it, flipping and zooming it to produce more examples per image.
                                zoom_range = 0.2,  #Data augmentation allows us to prevent overfitting of data into the training set.
                                horizontal_flip = True,
                                vertical_flip = True,
                                rotation_range = 40,
                                width_shift_range = 0.2,
                                height_shift_range = 0.2,
                                fill_mode = 'nearest'
                                )
test_data = ImageDataGenerator(rescale = 1/.255)

train_set = train_data.flow_from_directory('D:/amitavsir/bcd/datacompressed/train', #Image path
                                           target_size = (121,121), #The shape which we want to input our images in our model
                                           batch_size = 20,         #The batch size in mini batch gradient descent
                                           class_mode = 'categorical')   


test_set = test_data.flow_from_directory('D:/amitavsir/bcd/datacompressed/validation',
                                         target_size = (121,121),
                                         batch_size = 50,
                                         class_mode = 'categorical')

Found 962 images belonging to 4 classes.
Found 500 images belonging to 4 classes.


In [21]:
from sklearn.model_selection import KFold
import numpy as np

num_folds=10

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

fold_no = 1

In [22]:
for train, test in kfold.split(inputs, targets):
    #Building own model on top of trained network
    x = Conv2D(600, (1,1), activation = 'relu')(last)
    x = Flatten()(x)
    x = Dense(1028, activation = 'relu')(x)
    x = Dropout(rate = 0.3)(x)
    x = Dense(512, activation = 'relu')(x)
    x = Dropout(rate = 0.25)(x)
    x = Dense(4, activation = 'softmax')(x)

    #Compiling model
    model = Model(inputs = pre_trained_model.input, outputs = x, name = 'Predict')
    opt1 = Adam(learning_rate = 0.002)
    opt2 = RMSprop(learning_rate = 0.001)
    model.compile(optimizer = opt1 , loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(inputs[train], targets[train],
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity,        
              )
    
    # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

      # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/15
865/865 [==============================] - 81s 94ms/sample - loss: 1.6728 - acc: 0.4682
Epoch 2/15
865/865 [==============================] - 73s 85ms/sample - loss: 0.7284 - acc: 0.7272
Epoch 3/15
865/865 [==============================] - 73s 84ms/sample - loss: 0.5290 - acc: 0.8116
Epoch 4/15
865/865 [==============================] - 72s 83ms/sample - loss: 0.4487 - acc: 0.8173
Epoch 5/15
865/865 [==============================] - 74s 86ms/sample - loss: 0.2766 - acc: 0.9006
Epoch 6/15
865/865 [==============================] - 70s 81ms/sample - loss: 0.1998 - acc: 0.9306
Epoch 7/15
865/865 [==============================] - 72s 84ms/sample - loss: 0.2076 - acc: 0.9214
Epoch 8/15
865/865 [==============================] - 56s 64ms/sample - loss: 0.1592 - acc: 0.9434
Epoch 9/15
865/865 [==============================] - 53s 61ms/sample - loss: 0.2092 - acc: 0.9214
Epoch 10/15


In [23]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 4.714763336574908 - Accuracy: 39.175257086753845%
------------------------------------------------------------------------
> Fold 2 - Loss: 3.3151804638892104 - Accuracy: 54.63917255401611%
------------------------------------------------------------------------
> Fold 3 - Loss: 4.710722697149847 - Accuracy: 50.51546096801758%
------------------------------------------------------------------------
> Fold 4 - Loss: 3.9355133374532065 - Accuracy: 44.79166567325592%
------------------------------------------------------------------------
> Fold 5 - Loss: 5.69365914662679 - Accuracy: 43.75%
------------------------------------------------------------------------
> Fold 6 - Loss: 5.119016965230306 - Accuracy: 47.91666567325592%
------------------------------------------------------------------------
> Fold 7 - Loss